In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [18]:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]
LABEL = "medv"

training_set = pd.read_csv("boston_train.csv", skipinitialspace=True,skiprows=1,names=COLUMNS)

test_set = pd.read_csv("boston_test.csv",skipinitialspace=True,skiprows=1,names=COLUMNS)

prediction_set = pd.read_csv("boston_predict.csv", skipinitialspace=True,skiprows=1, names=COLUMNS)
print(test_set)

        crim    zn  indus    nox     rm    age      dis  tax  ptratio  medv
0    0.13587   0.0  10.59  0.489  6.064   59.1   4.2392  277     18.6  24.4
1    0.08664  45.0   3.44  0.437  7.178   26.3   6.4798  398     15.2  36.4
2    0.26938   0.0   9.90  0.544  6.266   82.8   3.2628  304     18.4  21.6
3    0.05302   0.0   3.41  0.489  7.079   63.1   3.4145  270     17.8  28.7
4    0.06860   0.0   2.89  0.445  7.416   62.5   3.4952  276     18.0  33.2
5    0.14231   0.0  10.01  0.547  6.254   84.2   2.2565  432     17.8  18.5
6    0.16760   0.0   7.38  0.493  6.426   52.3   4.5404  287     19.6  23.8
7    0.04301  80.0   1.91  0.413  5.663   21.9  10.5857  334     22.0  18.2
8    0.06905   0.0   2.18  0.458  7.147   54.2   6.0622  222     18.7  36.2
9    0.21719   0.0  10.59  0.489  5.807   53.8   3.6526  277     18.6  22.4
10   0.06076   0.0  11.93  0.573  6.976   91.0   2.1675  273     21.0  23.9
11   4.03841   0.0  18.10  0.532  6.229   90.7   3.0993  666     20.2  19.6
12   0.16902

In [9]:
feature_cols=[tf.contrib.layers.real_valued_column(k) for k in FEATURES]

In [10]:
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                          hidden_units=[9, 9],
                                          model_dir="/tmp/boston_model1")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001C8F8AB65F8>, '_is_chief': True, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_environment': 'local', '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_task_type': None, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_master': ''}


In [11]:
def input_fn(data_set):
    
    feature_cols = {k: tf.constant(data_set[k].values)
                  for k in FEATURES}
    labels = tf.constant(data_set[LABEL].values)
    return feature_cols, labels

In [2]:
regressor.fit(input_fn=lambda: input_fn(training_set), steps=5000)

NameError: name 'regressor' is not defined

In [13]:
ev = regressor.evaluate(input_fn=lambda: input_fn(test_set), steps=1)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-03-07-18:31:29
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-03-07-18:31:29
INFO:tensorflow:Saving dict for global step 15000: global_step = 15000, loss = 13.3096


In [14]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

Loss: 13.309587


In [15]:
y = regressor.predict(input_fn=lambda: input_fn(prediction_set))
# .predict() returns an iterator; convert to a list and print predictions
predictions = list(itertools.islice(y, 6))
print ("Predictions: {}".format(str(predictions)))

Predictions: [34.390099, 17.537806, 23.794096, 35.187134, 15.210985, 19.250265]
